## Assignment 4: Neural Networks
Shaan Verma

#### Importing required libraries

In [9]:
import torchvision.models as models
from torchvision import transforms
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import operator
%matplotlib inline
from torch import optim
from collections import namedtuple
from torch.autograd import Variable
from sklearn.linear_model import LogisticRegression
from __future__ import print_function, division
import pandas as pd
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import time
import os
import copy
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
%matplotlib inline

#### Getting AlexNet data

In [2]:
alexnet = models.alexnet(pretrained=True)

# Transform variable to convert images to the right size
transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])

#### Setting up the CIFAR-10 dataset

In [3]:
# Getting the CIFAR-10 dataset
dataset = CIFAR10(root='data/', download=True, transform=transform)
test_dataset = CIFAR10(root='data/', train=False, transform=transform)

# 10 Classes in CIFAR-10 dataset
classes = dataset.classes

torch.manual_seed(43)
val_size = 10000
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])

batch_size=100

train_loader = DataLoader(train_ds, batch_size, shuffle=False, num_workers=8, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size, num_workers=8, pin_memory=True)

Files already downloaded and verified


## Part B

In [4]:
# Getting 1000 possible classes for imagenet from downloaded file
with open("imagenet_classes.txt") as f:
    classes = eval(f.read())

# Temparary collections
holder = []
dic = {}
current = ''

# Iterates through eat batch of the testloader and gets output from AlexNet
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        out = alexnet(images)
        #print(out.shape)    


        for j in range(0,batch_size):
            sorted_out, indices = torch.sort(out,descending=True)
            percentage = F.softmax(out,dim=1)[j]*100
            results = [(classes[i.item()],percentage[i].item()) for i in indices[j][:5]]
            holder.append(results[0][0])

# Sorts all classification names
holder.sort()

# Populate the dictionary
for z in holder:
    if current != z:
        count = 1
        dic[z] = count
        current = z
    else:
        count = count + 1
        dic[z] = count 
        current = z
        
# Sorting the dictionary of classifications
sorted_d = dict( sorted(dic.items(), key=operator.itemgetter(1),reverse=True))

# Printing Top 10 classifications from alexNet
dicMax = 1
for i in sorted_d:
    print(dicMax,') ',i,':',sorted_d[i])
    dicMax = dicMax + 1
    if(dicMax > 10):
        break

1 )  moving van : 1049
2 )  fox squirrel, eastern fox squirrel, Sciurus niger : 604
3 )  sorrel : 340
4 )  container ship, containership, container vessel : 300
5 )  English foxhound : 246
6 )  Dandie Dinmont, Dandie Dinmont terrier : 232
7 )  thresher, thrasher, threshing machine : 227
8 )  Japanese spaniel : 226
9 )  milk can : 217
10 )  chain saw, chainsaw : 154


In [14]:
c_y_actu = pd.Series(['AP','AM','B','C','DR','DG','FG','H','S','T'], name='CIFAR-10')
c_y_pred = pd.Series(['MV','FES','SL','CCC','EF','DD','TTT','JS','MC','CC'], name='AlexNet')
df_confusion = pd.crosstab(c_y_actu, c_y_pred)
print(df_confusion)

AlexNet   CC  CCC  DD  EF  FES  JS  MC  MV  SL  TTT
CIFAR-10                                           
AM         0    0   0   0    1   0   0   0   0    0
AP         0    0   0   0    0   0   0   1   0    0
B          0    0   0   0    0   0   0   0   1    0
C          0    1   0   0    0   0   0   0   0    0
DG         0    0   1   0    0   0   0   0   0    0
DR         0    0   0   1    0   0   0   0   0    0
FG         0    0   0   0    0   0   0   0   0    1
H          0    0   0   0    0   1   0   0   0    0
S          0    0   0   0    0   0   1   0   0    0
T          1    0   0   0    0   0   0   0   0    0


## Part C


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


model = models.alexnet(pretrained=True)
for param in model.features.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = False


b =[]
for layer in model.classifier.children():
    b.append(layer)
b = b[:-5]

b.append(nn.Linear(4096, 10))
new_classifier = nn.Sequential(*b)
model.classifier = new_classifier

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0002, momentum=0.9)
save_path = './AlexNetc.pth'
best_acc = 0.0
train_steps = len(train_loader)
epochs = 2

print("Start Training")
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    train_bar = tqdm(train_loader)
    for i, data in enumerate(train_bar):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1,
                                                                     epochs,
                                                                     loss)


    model.eval()
    acc = 0.0
    with torch.no_grad():
        val_bar = tqdm(val_loader)
        for data in val_bar:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            acc += torch.eq(predicted, labels).sum().item()
    val_accurate = acc / val_size
    print('[epoch %d] train_loss: %.3f  val_accuracy: %.3f' %
              (epoch + 1, running_loss / train_steps, val_accurate))

    if val_accurate > best_acc:
        best_acc = val_accurate
        torch.save(model.state_dict(), save_path)

# Evaluating using the train partition         
print('Finished Training')
print('Start Testing with train partition')
model.load_state_dict(torch.load(save_path))
model.eval()
acc = 0.0
with torch.no_grad():
    test_bar = tqdm(train_loader)
    for data in test_bar:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        acc += torch.eq(predicted, labels).sum().item()
test_accurate = acc / 10000
print('test_accuracy: %.3f' %(test_accurate))

# Evaluating using the validation partition 
print('Start Testing with valadation partition')
model.load_state_dict(torch.load(save_path))
model.eval()
acc = 0.0
with torch.no_grad():
    test_bar = tqdm(val_loader)
    for data in test_bar:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        acc += torch.eq(predicted, labels).sum().item()
test_accurate = acc / 10000
print('test_accuracy: %.3f' %(test_accurate))


# Evaluating using the test partition 
print('Start Testing with test partition')
model.load_state_dict(torch.load(save_path))
model.eval()
acc = 0.0
with torch.no_grad():
    test_bar = tqdm(test_loader)
    for data in test_bar:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        acc += torch.eq(predicted, labels).sum().item()
test_accurate = acc / 10000
print('test_accuracy: %.3f' %(test_accurate))



  0%|          | 0/400 [00:00<?, ?it/s]

Start Training


100%|██████████| 100/100 [01:37<00:00,  1.03it/s]


[epoch 1] train_loss: 0.883  val_accuracy: 0.736


100%|██████████| 100/100 [02:07<00:00,  1.28s/it]


...

test_accuracy: 3.050
Start Testing with valadation partition


  0%|          | 0/100 [00:00<?, ?it/s]

test_accuracy: 0.751
Start Testing with test partition


100%|██████████| 100/100 [02:05<00:00,  1.26s/it]

test_accuracy: 0.750


## Part D

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


model = models.alexnet(pretrained=True)
for param in model.features.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = False


b =[]
for layer in model.classifier.children():
    b.append(layer)
b = b[:-2]

b.append(nn.Linear(4096, 10))
new_classifier = nn.Sequential(*b)
model.classifier = new_classifier

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
save_path = './AlexNetc.pth'
best_acc = 0.0
train_steps = len(train_loader)
epochs = 2

print("Start Training")
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    train_bar = tqdm(train_loader)
    for i, data in enumerate(train_bar):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1,
                                                                     epochs,
                                                                     loss)


    model.eval()
    acc = 0.0
    with torch.no_grad():
        val_bar = tqdm(val_loader)
        for data in val_bar:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            acc += torch.eq(predicted, labels).sum().item()
    val_accurate = acc / val_size
    print('[epoch %d] train_loss: %.3f  val_accuracy: %.3f' %
              (epoch + 1, running_loss / train_steps, val_accurate))

    if val_accurate > best_acc:
        best_acc = val_accurate
        torch.save(model.state_dict(), save_path)

            
print('Finished Training')
print('Start Testing with training partition')
model.load_state_dict(torch.load(save_path))
model.eval()
acc = 0.0
with torch.no_grad():
    test_bar = tqdm(train_loader)
    for data in test_bar:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        acc += torch.eq(predicted, labels).sum().item()
test_accurate = acc / 10000
print('test_accuracy: %.3f' %(test_accurate))


print('Finished Training')
print('Start Testing with validation partition')
model.load_state_dict(torch.load(save_path))
model.eval()
acc = 0.0
with torch.no_grad():
    test_bar = tqdm(val_loader)
    for data in test_bar:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        acc += torch.eq(predicted, labels).sum().item()
test_accurate = acc / 10000
print('test_accuracy: %.3f' %(test_accurate))


print('Start Testing with test partition')
model.load_state_dict(torch.load(save_path))
model.eval()
acc = 0.0
with torch.no_grad():
    test_bar = tqdm(test_loader)
    for data in test_bar:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        acc += torch.eq(predicted, labels).sum().item()
test_accurate = acc / 10000
print('test_accuracy: %.3f' %(test_accurate))

  0%|          | 0/400 [00:00<?, ?it/s]

Start Training


100%|██████████| 100/100 [02:15<00:00,  1.35s/it]


[epoch 1] train_loss: 65.055  val_accuracy: 0.625


100%|██████████| 100/100 [02:20<00:00,  1.40s/it]


...

test_accuracy: 2.771
Finished Training
Start Testing with validation partition


  0%|          | 0/100 [00:00<?, ?it/s]

test_accuracy: 0.690
Start Testing with test partition


100%|██████████| 100/100 [02:13<00:00,  1.34s/it]

test_accuracy: 0.689
